In [1]:
import pandas as pd
import numpy as np
import os
import grid2op
from grid2op.Chronics import ChangeNothing
from chronix2grid.kpi.Generator_parameter_checker import EnergyMix_AprioriChecker

In [2]:
input_path = os.path.join("..", "example", "input", "generation", "case118_l2rpn_wcci")

In [3]:
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))

In [4]:
df

,name,Pmax,Pmin,type,bus,max_ramp_up,max_ramp_down,min_up_time,min_down_time,marginal_cost,shut_down_cost,start_cost,zone,generator,x,y,V
0,gen_3_0,33.6,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,0,62,-101,142.1
1,gen_3_1,33.6,0.0,wind,3,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,70,62,-101,142.1
2,gen_7_2,400.0,0.0,nuclear,7,2.8,2.8,2016.0,2016.0,35.0,0,27728.7,R1,2,74,-293,355.4
3,gen_9_3,300.0,0.0,thermal,9,5.8,5.8,12.0,12.0,50.0,0,3392.3,R1,3,74,-390,355.4
4,gen_10_4,37.3,0.0,solar,10,NaN,NaN,0.0,0.0,0.0,0,0.0,R1,69,116,-104,142.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,gen_106_57,67.2,0.0,wind,106,NaN,NaN,0.0,0.0,0.0,0,0.0,R3,48,988,-582,142.1
58,gen_110_58,150.0,0.0,thermal,110,4.3,4.3,24.0,24.0,50.0,0,15319.4,R3,50,900,-707,142.1
59,gen_111_59,67.2,0.0,wind,111,NaN,NaN,0.0,0.0,0.0,0,0.0,R3,51,987,-699,142.1
60,gen_112_60,100.0,0.0,thermal,112,2.8,2.8,24.0,24.0,50.0,0,18016.4,R1,52,203,-276,142.1


In [5]:
pmaxs = df.groupby(["type"])["Pmax"].sum()
types = pmaxs.index.to_numpy()
pmaxs = pmaxs.to_numpy()

In [6]:
types, pmaxs, pmaxs.sum()

(array(['hydro', 'nuclear', 'solar', 'thermal', 'wind'], dtype=object),
 array([1750. , 1200. ,  746.4, 3100. ,  672. ]),
 7468.4)

In [7]:
def get_apriori_energy_mix(capacity_factor, average_load):
  apriori_energy_mix = capacity_factor * pmaxs / average_load
  total = np.nansum(apriori_energy_mix)
  if total > 100:
    print("WARNING: apriori energy mix w/o thermal exceeds average load!")
    apriori_energy_mix[0] -= (total - 100)
    apriori_energy_mix[3] = 0
  else:
    apriori_energy_mix[3] = 100 - total
  return apriori_energy_mix


In [8]:
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
capacity_factor_df = pd.DataFrame(data=capacity_factor, columns=['capacity_factor'], index=types)

apriori_energy_mix = get_apriori_energy_mix(capacity_factor, average_load)

In [9]:
grid_path = os.path.join(input_path, "grid.json")

In [10]:
env118_withoutchron = grid2op.make(
    "rte_case118_example",
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    chronics_class=ChangeNothing, # tell it to change nothing (not the most usable environment...)
)

/home/gaetan/.local/lib/python3.8/site-packages/grid2op/MakeEnv/Make.py:305: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


In [11]:
Target_EM_percentage=pd.DataFrame(data=[18, 40, 4, 30, 6], columns=['target_energy_mix'], index=types)

#Variable used to anticipate the energy mix a priori. Update them after chronics generation if too different
PeakLoad = 4200 #expected peak load

Capacity_df=EnergyMix_AprioriChecker(env118_withoutchron, Target_EM_percentage, PeakLoad, average_load, capacity_factor_df)
Capacity_df["Apriori_energy_mix"]

type
hydro      18.750000
nuclear    40.714286
solar       3.998571
thermal    30.537143
wind        6.000000
Name: Apriori_energy_mix, dtype: float64

In [12]:
apriori_energy_mix

array([18.75      , 40.71428571,  3.99857143, 30.53714286,  6.        ])